In [1]:
from scipy.spatial.transform import Rotation
import numpy as np
import os
os.chdir('../')
from math import sin, cos

SMPL_PARENT = {
    1: 0, 2: 0, 3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 10: 7, 
    11: 8, 12: 9, 13: 9, 14: 9, 15: 12, 16: 13, 17: 14, 18: 16, 19: 17, 20: 18, 
    21: 19, 22: 20, 23: 21}

In [ ]:

def _to_skew_matrix(v):
    r""" Compute the skew matrix given a 3D vectors.

    Args:
        - v: Array (3, )

    Returns:
        - Array (3, 3)

    """
    vx, vy, vz = v.ravel()
    return np.array([[0, -vz, vy],
                    [vz, 0, -vx],
                    [-vy, vx, 0]])
    
def _rvec_to_rmtx(rvec):
    r''' apply Rodriguez Formula on rotate vector (3,)

    Args:
        - rvec: Array (3,)

    Returns:
        - Array (3, 3)
    '''
    rvec = rvec.reshape(3, 1)

    norm = np.linalg.norm(rvec)
    theta = norm
    r = rvec / (norm + 1e-5)

    skew_mtx = _to_skew_matrix(r)

    return cos(theta)*np.eye(3) + \
           sin(theta)*skew_mtx + \
           (1-cos(theta))*r.dot(r.T)
           
def body_pose_to_body_RTs(jangles, tpose_joints):
    r""" Convert body pose to global rotation matrix R and translation T.
    
    Args:
        - jangles (joint angles): Array (Total_Joints x 3, )
        - tpose_joints:           Array (Total_Joints, 3)

    Returns:
        - Rs: Array (Total_Joints, 3, 3)
        - Ts: Array (Total_Joints, 3)
    """

    jangles = jangles.reshape(-1, 3)
    total_joints = jangles.shape[0]
    assert tpose_joints.shape[0] == total_joints

    Rs = np.zeros(shape=[total_joints, 3, 3], dtype='float32')
    Rs[0] = _rvec_to_rmtx(jangles[0,:])

    Ts = np.zeros(shape=[total_joints, 3], dtype='float32')
    Ts[0] = tpose_joints[0,:]

    for i in range(1, total_joints):
        Rs[i] = _rvec_to_rmtx(jangles[i,:])
        Ts[i] = tpose_joints[i,:] - tpose_joints[SMPL_PARENT[i], :]
    
    return Rs, Ts


In [ ]:


def _construct_G(R_mtx, T):
    batch_size, total_bones = R_mtx.shape[:2]

    G = np.zeros(shape=(batch_size, total_bones, 4, 4),
                    dtype=R_mtx.dtype, device=R_mtx.device)
    G[:, :, :3, :3] = R_mtx
    G[:, :, :3, 3] = T
    G[:, :, 3, 3] = 1.0

    return G

def forward(dst_Rs, dst_Ts, cnl_gtfms):
    dst_gtfms = np.zeros_like(cnl_gtfms)

    local_Gs = _construct_G(dst_Rs, dst_Ts)    
    dst_gtfms[:, 0, :, :] = local_Gs[:, 0, :, :]

    for i in range(1, 24):
        dst_gtfms[:, i, :, :] = np.matmul(
                                    dst_gtfms[:, SMPL_PARENT[i], 
                                                :, :].clone(),
                                    local_Gs[:, i, :, :])

    dst_gtfms = dst_gtfms.view(-1, 4, 4)
    inv_dst_gtfms = np.linalg.inv(dst_gtfms)
    
    cnl_gtfms = cnl_gtfms.view(-1, 4, 4)
    f_mtx = np.matmul(cnl_gtfms, inv_dst_gtfms)
    f_mtx = f_mtx.view(-1, 24, 4, 4)

    scale_Rs = f_mtx[:, :, :3, :3]
    Ts = f_mtx[:, :, :3, 3]

    return scale_Rs, Ts

In [ ]:
import pickle

# pose
with open("data/data_prepared/CoreView_377_arah/models/000000.npz", 'rb') as f:   
    model_pnz = pickle.load(f)
root_orient = model_pnz['root_orient']
pose_body = model_pnz['pose_body']
pose_hand = model_pnz['pose_hand']
pose = np.concatenate(root_orient, pose_body, pose_hand).reshape(-1, 3)

# tpose joint
with open("data/data_prepared/CoreView_377/1/canonical_joints.pkl", "r") as f:
    model_pkl = pickle.load(f)
tpose_joints = model_pkl['joints']
f_mtx = forward(pose, tpose_joints)
